In [5]:
import MetaTrader5 as mt5
import pandas as pd 
mt5.initialize()

True

In [6]:
def RolarTravaAltaPut(ativosLista, comprei, vendi ):
    # [travaFeitaC (comprada) , travaFeitaV (vendida), travaNovaC, travaNovaV]
    travaFeitaC = mt5.symbol_info(ativosLista[0])
    travaFeitaV = mt5.symbol_info(ativosLista[1])
    travaNovaC  = mt5.symbol_info(ativosLista[2])
    travaNovaV  = mt5.symbol_info(ativosLista[3])
    ativoBase = travaFeitaC.basis
    ativoBaseInfo = mt5.symbol_info(ativoBase)

    lin,col = (3,10)
    matriz = [[0 for i in range(col)] for j in range(lin)]
    ###########################################
    # coluna 0 - ativos
    matriz[0][0] = travaFeitaC.name 
    matriz[1][0] = travaFeitaV.name
    matriz[2][0] = 'Ʃ'

    # coluna 1 - strike feitos
    matriz[0][1] = travaFeitaC.option_strike
    matriz[1][1] = travaFeitaV.option_strike
    matriz[2][1] = matriz[1][1] - matriz[0][1]

    # coluna 2 - valores da trava feita
    matriz[0][2] = round(comprei,2)
    matriz[1][2] = round(vendi,2)
    matriz[2][2] = round(matriz[1][2] + matriz[0][2], 2)

    # coluna 3 - valores para desmontar a trava feita
    matriz[0][3] = round(travaFeitaC.bid,2)
    matriz[1][3] = round(-travaFeitaV.ask,2)
    matriz[2][3] = round(matriz[1][3] + matriz[0][3], 2)

    # coluna 4 - ativos novos
    matriz[0][4] = travaNovaC.name 
    matriz[1][4] = travaNovaV.name
    matriz[2][4] = 'Ʃ'

    
    # coluna 5 - strikes novos
    matriz[0][5] = round(travaNovaC.option_strike,2)
    matriz[1][5] = round(travaNovaV.option_strike,2)
    matriz[2][5] = round(matriz[1][1] - matriz[0][1], 2)

    # coluna 6 - valores para montar nova trava
    matriz[0][6] = -travaNovaC.ask
    matriz[1][6] = travaNovaV.bid
    if matriz[1][6] == 0:
        fff = input('Entre o valor do Bid ' + travaNovaV.name + ': ')
        matriz[1][6] = float(fff)
    matriz[2][6] = matriz[1][6] + matriz[0][6]

    # coluna 7 - ..
    matriz[0][7] = '/'
    matriz[1][7] = '/'
    matriz[2][7] = 'resultado final:'

    matriz[2][8] = (matriz[2][2] + matriz[2][3] + matriz[2][6])  

    # coluna 8 - Basis
    matriz[0][9] = ativoBase
    matriz[1][9] = ativoBaseInfo.last
    matriz[2][9] = ativoBaseInfo.price_change

   

    df = pd.DataFrame(matriz, columns = ['Ativo', 'Strike', '$ Montei', '$ Desmontar' ,'Ativo Novo','Strike Novo', '$ montar novo' , '...','..' ,  'Basis'])
    print(df)
    nome_arq = 'RolarTravaAltaPut_' + ativoBase  + '.xlsx'
    df.to_excel(nome_arq)


In [47]:
RolarTravaAltaPut(['BBSES220', 'BBSES235', 'BBSET220', 'BBSET235'], -0.15, 0.4)

      Ativo  Strike  $ Montei  $ Desmontar Ativo Novo  Strike Novo  \
0  BBSES220    22.0     -0.15         0.08   BBSET220         22.0   
1  BBSES235    23.5      0.40        -0.00   BBSET235         23.5   
2         Ʃ     1.5      0.25         0.08          Ʃ          1.5   

   $ montar novo               ...    ..   Basis  
0          -0.40                 /  0.00   BBSE3  
1           0.95                 /  0.00   22.62  
2           0.55  resultado final:  0.88 -1.0066  


In [48]:
RolarTravaAltaPut(['B3SAS805', 'B3SAS17',  'B3SAT148', 'B3SAT170'], -0.1, 0.66)

      Ativo  Strike  $ Montei  $ Desmontar Ativo Novo  Strike Novo  \
0  B3SAS805   14.66     -0.10         0.02   B3SAT148        14.59   
1   B3SAS17   16.66      0.66        -0.64   B3SAT170        16.51   
2         Ʃ    2.00      0.56        -0.62          Ʃ         2.00   

   $ montar novo               ...    ..   Basis  
0          -0.19                 /  0.00   B3SA3  
1           0.82                 /  0.00    16.1  
2           0.63  resultado final:  0.57 -1.0449  
